### Hyperparameters
| ** Hyper-parameter ** | ** Variants ** |
| -------------------- | ---------------- |
| Non-linearity | linear, tanh, sigmoid, ReLU, VLReLU, RReLU, PReLU, ELU, maxout, APL, combination |
| Batch Normalization | Before Non-Linearity, After Non-Linearity |
| BN + Non-Linearity | linear, tanh, sigmoid, ReLU, VLReLU, RReLU, PReLU, ELU, maxout |
| Pooling | max, average, stochastic, max+average, stided convolution |
| Pooling Window Size | 3x3, 2x2, 3x3 with Zero-Paddding |
| Learning Rate Decay Policy | step, Square, square-root, linear |
| Colorspace & Pre-Processing | RGB, HSV, YCrCb, grayscale, learned, CLAHE, histogram equalized |
| Classifier Design | pooling-FC-FC-clf, SPP-FC-FC-clf, pooling-conv-conv-clf-avepool, pooling-conv-conv-avepool-clf |
| Network Width | 1/4, 1/2root2, 1/2, 1/root2, 1, root2, 2, 2root2, 4, 4root2 |
| Input Image Size | 64, 96, 128, 180, 224 |
| Dataset Size | 200K, 400K, 600K, 800K, 1200K(full) |
| Batch Size | 1, 32, 64, 128, 256, 512, 1024 |
| Percentage of noisy data | 0, 5%, 10%, 15%, 32% |
| Using Bias | yes/no |

In [1]:
import json
import glob
import sys

import numpy as np
import pandas as pd


import matplotlib.pylab as plt
import seaborn as sns
sns.despine()


In [2]:
IMG_HELP_PATH = '/Users/shankar/dev/code/ds/studies/data_science/trading/honchar'
import os
import sys
sys.path.append(os.path.abspath(IMG_HELP_PATH))
from common.preprocessing import split_train_test_in_unison
from common.datasets.bitcoin import BitCoinPreprocess

In [3]:
data = pd.read_csv('../datasets/bitcoin/krakenUSD_1-min_data_2014-01-07_to_2017-05-31.csv')[-10000:]
data = data.dropna()

In [4]:
STEP = 1
FORECAST = 10
WINDOW = 30

In [5]:
btcPreprocess = BitCoinPreprocess()
X, Y = btcPreprocess.preprocess(data, WINDOW, STEP, FORECAST)

list index out of range


In [6]:
X_train, X_test, Y_train, Y_test = split_train_test_in_unison(X, Y)

In [7]:
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras import losses

/anaconda3/envs/dlcv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
def experiment(params):
    print('Trying {}'.format(params))
    try:
        input_ = Input(shape=(len(X_train[0]), ), name='input_')
        X = Dense(params['units1'], activation=params['activation'])(input_)
        X = Dense(params['units2'], activation=params['activation'])(X)
        X = Dense(params['units3'], activation=params['activation'])(X)
        
        output = Dense(1, activation='linear', name='out')(X)
        model = Model(inputs[input_], outputs=[output])
        
        opt = Adam(lr=params['lr'])
        model.compile(optimizer=opt, loss=params['loss'])
        
        H = model.fit(X_train, Y_train, epochs=5, batch_size=256, verbose=1, validation_data=(X_test, Y_test), shuffle=True)
        
        pred = model.predict(X_test)
        
    except Exception as e: 
        print(e)
        return {'loss': 999999, 'status': STATUS_OK}
    
    predicted = pred
    original = Y_test
    
    if(np.isnan(mse)):
        print('NaN Happened')
        print('-' * 10)
        return {'loss': 999999, 'status': STATUS_OK}
    
    print(mse)
    print('-' * 10)
        
    sys.stdout.flush()
    return {'loss': mse, 'status': STATUS_OK}
        

In [9]:
space = {
    'window': hp.choice('window', [30, 60, 120, 180]),
    'units1': hp.choice('units1', [64, 512]),
    'units2': hp.choice('units2', [64, 512]),
    'units3': hp.choice('units3', [64, 512]),
    'lr': hp.choice('lr', [0.01, 0.001, 0.0001]),
    'activation': hp.choice('activation', ['relu', 'sigmoid', 'tanh', 'linear']),
    'loss': hp.choice('loss', [losses.logcosh, losses.mse, losses.mae, losses.mape])
}
trials = Trials()
best = fmin(experiment, space, algo=tpe.suggest, max_evals=50, trials=trials)
print('best')
print(best)

Trying {'activation': 'tanh', 'loss': <function mean_absolute_error at 0x1a2514c158>, 'lr': 0.01, 'units1': 64, 'units2': 512, 'units3': 64, 'window': 120}
name 'Input' is not defined
Trying {'activation': 'tanh', 'loss': <function logcosh at 0x1a2514c488>, 'lr': 0.0001, 'units1': 64, 'units2': 64, 'units3': 64, 'window': 120}
name 'Input' is not defined
Trying {'activation': 'linear', 'loss': <function mean_absolute_error at 0x1a2514c158>, 'lr': 0.0001, 'units1': 512, 'units2': 512, 'units3': 512, 'window': 60}
name 'Input' is not defined
Trying {'activation': 'relu', 'loss': <function mean_absolute_percentage_error at 0x1a2514c1e0>, 'lr': 0.0001, 'units1': 512, 'units2': 64, 'units3': 64, 'window': 30}
name 'Input' is not defined
Trying {'activation': 'sigmoid', 'loss': <function mean_absolute_percentage_error at 0x1a2514c1e0>, 'lr': 0.01, 'units1': 64, 'units2': 512, 'units3': 512, 'window': 120}
name 'Input' is not defined
Trying {'activation': 'relu', 'loss': <function logcosh at 